![logo](https://resolvephilly.org/themes/custom/resolvephl-ci/logo.svg)

# Scraping and standardizing Pennsylvania Act 146 Individual Reports

**Author:** Julie Christie | Director of Data & Impact

**Partnering Team:** Our Kids

**Date:** March 28, 2024

## Background

Pennsylvania publishes reports on child fatalities and near fatalities as a result of child abuse. The reports include the age, sex, county, and date of the incident, as well as whether the family was known to the local department of human/family services in the 16 months preceding the incident. The detailed reports are organized online in a way that can be scraped for some, but not all of the information. The first page of the `.pdf` reports contain the rest of the necessary information.

### Goal of Analysis

Specifically, Resolve is looking to understand the frequency at which children who experience abuse that results in their death/near death are already known to the system. We are exploring these rates at the county level to understand what the statewide trend is, and whether Philadelphia exceeds that trend.

### Glossary

-   **Act 146** -- *"Act 146 of 2006 went into effect on May 8, 2007. A major provision of this law requires that the department prepare a non-identifying summary for the governor and the General Assembly of findings for each case of substantiated child abuse or neglect that has resulted in a child fatality or near fatality."*
-   **Near fatality** -- *Definition TKTK*
-   **DA De-certification** -- *This gets assigned to a report when the District Attorney determines that the incident was not a result of child abuse.*

### Data

-   [Child Fatality/Near Fatality Quarterly Reports](https://www.pa.gov/en/agencies/dhs/resources/data-reports/quarterly-summaries-child-abuse.html) --- A collection of brief summaries of fatalities/near fatalities of children due to abuse. | No metadata available

### Tools

-   [Python](python.org) -- *Base code to facilitate scraping*
-   [Pandas](https://pandas.pydata.org/) -- *More robust data anlysis*
-   [Regex](https://developers.google.com/edu/python/regular-expressions) -- *Regular Expressions, or Regex, to parse out patterns of characters*
-   [PDF Plumber](https://github.com/jsvine/pdfplumber) -- *Parse information from .pdf files*
-   [Excel](https://www.microsoft.com/en-us/microsoft-365/p/excel/cfq7ttc0hr4r?activetab=pivot:overviewtab) -- *Clean and analyze tabulated data*

### Limitations
- A "certifying pysician" makes an individual call on whether a child's death/near death is the result of abuse, meaning that human error may result in cases not being documented in these reports
- Child fatalities and near fatalities as a result of abuse are an incredibly small and extreme subset of the overall abuse that children face. This analysis does not constitute a full picture, but rather is a snapshot of what the state deemed the most egregious cases.
- These quarterly reports may not contain all instances. A previous scrape of individual reports rendered about 2,400 reports. This scrape yielded TKTK summaries.

## Cleaning

1. Download all the reports from the Pennsylvania DHS site.

### Parse single report

In [ ]:
pip install pdfplumber

In [ ]:
pip install pandas

1. Import the needed libraries

In [1]:
import re #regular expressions

import pdfplumber
import pandas as pd
from collections import namedtuple

1. Create the column names for the data that you are extracting from the pdf.

In [7]:
Line = namedtuple('Line', 'fatality county age sex date cause perpetrator indicated_date known_to_agency')

2. Create a list of all 67 counties in PA to match with the different headers of the reports.

In [4]:
pa_counties = ["Adams", "Allegheny", "Armstrong", "Beaver", "Bedford", "Berks", "Blair", "Bradford", "Bucks", "Butler", "Cambria", "Cameron", "Carbon", "Centre", "Chester", "Clarion", "Clearfield", "Clinton", "Columbia", "Crawford", "Cumberland", "Dauphin", "Delaware", "Elk", "Erie", "Fayette", "Forest", "Franklin", "Fulton", "Greene", "Huntingdon", "Indiana", "Jefferson", "Juniata", "Lackawanna", "Lancaster", "Lawrence", "Lebanon", "Lehigh", "Luzerne", "Lycoming", "McKean", "Mercer", "Mifflin", "Monroe", "Montgomery", "Montour", "Northampton", "Northumberland", "Perry", "Philadelphia", "Pike", "Potter", "Schuylkill", "Snyder", "Somerset", "Sullivan", "Susquehanna", "Tioga", "Union", "Venango", "Warren", "Washington", "Wayne", "Westmoreland", "Wyoming", "York"]

3. All of the documents are organized by listing whether the case was a fatality as nested headers. The below code sets up a regex function that identifies the headers.

In [13]:
fatality_re = re.compile(r'(Fatalities|Near Fatalities)')
line_re = re.compile(r'\d{1,2}((\.|(\)))|\.(\)))\s')

In [12]:
line_re.search('')

4. Set the file for your search

In [6]:
file = 'act_33_quarterly/1st Quarter Summaries of Child Fatalities Near Fatalities (1).pdf'

5. Parse out the data in the report

In [ ]:
lines = []

with pdfplumber.open(file) as pdf:
    pages = pdf.pages
    for page in pdf.pages:
        text = page.extract_text()
        for line in text.split('(\n|\r)\d{1,2}((\.|(\)))|\.(\)))\s'):
            print(line)
            fatality_set = fatality_re.search(line)
            if fatality_set:
                fatality = fatality_set.group(1)

            elif line.startswith(tuple(pa_counties)):
                county = line

            elif line_re.search(line):
                items = line.split()
                lines.append(Line(vend_no, vend_name, doctype, *items))
